In [55]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from time import sleep
import time
import pandas as pd
import numpy as np

In [56]:
# Setup Selenium options
options = Options()

# Do NOT use headless mode initially to see the CAPTCHA/challenge
# options.add_argument("--headless")  # You can enable this later
options.add_argument("--disable-gpu")
# options.add_argument("--no-sandbox")
# options.add_argument("--disable-dev-shm-usage")

# Initialize the driver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

name_list = []
date_list = []
price_list = []
shop_code_list = []
JAN_code_list = []
brand_list = []
series_title_list = []
char_name_list = []
sculptor_list = []
size_list = []
gcode_list = []
pages = 1

while pages <= 2 :
    base_URL = "https://www.amiami.com/eng/search/list/?s_condition_flg=1&s_st_condition_flg=1&s_sortkey=preowned&pagecnt="
    final_URL = base_URL + str(pages)
    driver.get(final_URL)
    time.sleep(10)

    page_html = driver.page_source
    soup = BeautifulSoup(page_html, 'html.parser')
    items = soup.find(class_='new-items__inner').findAll('a', href=True)

    for item in items:
        gcode = item['href']  # Extract the href attribute (the URL)
        gcode_list.append(gcode)
    pages += 1

# driver.quit()
        

In [57]:
base_link = "https://www.amiami.com"
count = 0
# for gcode_url in gcode_list:
figure_url = base_link + gcode_list[19]
# doubleprice_URL = "https://www.amiami.com/eng/detail/?gcode=FIGURE-156533-R"

driver.get(figure_url)
# driver.get(doubleprice_URL)
# time.sleep(10)

figure_html = driver.page_source
driver.quit()

figure_soup = BeautifulSoup(figure_html, "html.parser")
item_details = figure_soup.findAll(class_ = ['item-detail__right', 'item-about'])

In [58]:
# price1 = figure_soup.findAll(class_ = ["item-detail__price"])
# price = figure_soup.find(class_ = ["item-detail__price"])
# temp = price
# doubleprice = temp
# print(doubleprice.text)

In [63]:
item_details

[<div class="item-detail__right"><div class="wrapper"><ul class="item-detail__tag-list pconly"><li class="item-detail__tag-list__line item-detail__tag-list__line_sale" style="display: none;">Sale</li><li class="item-detail__tag-list__line item-detail__tag-list__line_limited" style="display: none;">Limited</li><li class="item-detail__tag-list__item item-detail__tag-list__item_preowned" style="display: none;">Pre-owned</li><li class="item-detail__tag-list__item item-detail__tag-list__item_preorder" style="display: none;">Pre-order</li><li class="item-detail__tag-list__item item-detail__tag-list__item_backorder" style="display: none;">Back-order</li><li class="item-detail__tag-list__item" style="display: none;">Order Closed</li></ul><h2 class="item-detail__section-title"></h2><p class="item-detail__note"></p><p class="item-detail__jpn"></p><p class="item-detail__brand"></p><!-- --><!-- --><p class="item-detail__point" style="display: none;"> points rewarded</p><p class="item-detail__deliv

In [59]:
item_about = figure_soup.findAll(class_ = ["item-about__data-text"], limit = 10)
        
temp_list = []
for i in item_about:
    temp_list.append(i.text)
    
fig_name = figure_soup.find(class_ = ["item-detail__section-title"]).text
fig_release_date = temp_list[0]
fig_price = temp_list[1]
fig_shop_code = temp_list[2]
fig_JAN_code = temp_list[3]
fig_brand = temp_list[4]
fig_series_title = temp_list[5]
fig_char_name = temp_list[6]
if temp_list[7] == "The maximum purchase quantity for this item is 1 per account/shipping address.":
    fig_sculptor = np.nan
else:
    fig_sculptor = temp_list[7]    

IndexError: list index out of range

In [36]:
for i in item_about:
    print(i.text)

Oct-2012
8,580 JPY
FIG-MOE-5872-RA7E
4934054782499
Kotobukiya
4-Leaves
Infinite Stratos (IS)"Infinite Stratos (IS)" Series
Charlotte Dunois
Ryo Hashimoto, Takanari 
The maximum purchase quantity for this item is 1 per account/shipping address.


In [37]:
print(fig_name)

(Pre-owned ITEM:B/BOX:B)Infinite Stratos - Charlotte Dunois -Bunny Style- 1/7 Complete Figure(Released)


In [38]:
item_specs = figure_soup.find(class_ = ["more"])
        
specs = item_specs.get_text("|")    # Joins the bits of text together using | & clears out the <br/> tags
splitSpecs = specs.split("|")       # Separates the long string into bits of strings so we can access stuff we want easier
flag = 0
for s in splitSpecs:
    print(s)
    if flag != 1:
        if "Size" in s:
            # fig_size = ""
            fig_size = s
            flag = 1
        else:
            fig_size = temp_list[8]

Sculptor: Ryo Hashimoto, Takanari
Here comes everyone's beloved character, Charlotte Dunois from Infinite Stratos!
She has been sculpted based on the illustration drawn by Tomoyasu Kurashima, the character designer of IS.
For making Charlotte's beautiful legs look even more beautiful, real fishnet has been used for the fishnet stockings!
The rabbit on the waist is attached by magnet and it can be removed as well.
Display her in your favorite way!


In [39]:
fig_size

'Ryo Hashimoto, Takanari\xa0'

In [25]:
name_list.append(fig_name)
date_list.append(fig_release_date)
price_list.append(fig_price)
shop_code_list.append(fig_shop_code)
JAN_code_list.append(fig_JAN_code)
brand_list.append(fig_brand)
series_title_list.append(fig_series_title)
char_name_list.append(fig_char_name)
sculptor_list.append(fig_sculptor)
size_list.append(fig_size)

In [26]:
df_figure = pd.DataFrame(
    {'Name': name_list,
     'Release Date': date_list,
     'Price': price_list,
     'Shop Code': shop_code_list,
     'JAN Code': JAN_code_list,
     'Brand': brand_list,
     'Series Title': series_title_list,
     'Character Name': char_name_list,
     'Sculptor': sculptor_list,
     'Size Specifications': size_list
    })

In [27]:
df_figure

,Name,Release Date,Price,Shop Code,JAN Code,Brand,Series Title,Character Name,Sculptor,Size Specifications
0,[(Pre-owned ITEM:B/BOX:B)Comic Aun Alice illus...,[Nov-2020],"[16,280 JPY]",[FIGURE-060005-R149],[4562283281713],[[[SkyTube]]],[[[Comic Aun]]],[[[Alice]]],"[HIRO, [ ]]",Size: Main Part: Approx. H25.5cm
1,[(Pre-owned ITEM:B/BOX:B)Comic Aun Alice illus...,[Nov-2020],"[16,280 JPY]",[FIGURE-060005-R149],[4562283281713],[[[SkyTube]]],[[[Comic Aun]]],[[[Alice]]],"[HIRO, [ ]]",Size: Main Part: Approx. H25.5cm
2,[(Pre-owned ITEM:B/BOX:B)Comic Aun Alice illus...,Nov-2020,"16,280 JPY",FIGURE-060005-R149,4562283281713,SkyTube,Comic Aun,Alice,HIRO,Size: Main Part: Approx. H25.5cm
